# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [66]:
import warnings
warnings.filterwarnings('ignore')

In [67]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [68]:
#!pip install pandas

In [69]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [70]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [71]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [72]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [73]:
prompt = ChatPromptTemplate.from_template(
    "Tell me about the following product: {product_description}. What are its key features, benefits, and potential use cases?"
)

In [74]:
chain = LLMChain(llm=llm, prompt=prompt)

In [75]:
product = "wireless noise-cancelling headphones"
chain.run(product)

'Wireless noise-cancelling headphones are a type of headphones that use active noise control technology to reduce ambient noise from the surrounding environment. This allows the user to enjoy their music or audio content without being disturbed by external sounds.\n\nKey features of wireless noise-cancelling headphones include:\n\n1. Bluetooth connectivity: These headphones are wireless and can be connected to devices such as smartphones, tablets, or laptops via Bluetooth, allowing for easy and tangle-free usage.\n\n2. Active noise cancellation: This technology uses built-in microphones to pick up external sounds and then generates anti-noise to cancel out the unwanted sounds, creating a quiet environment for the user.\n\n3. Long battery life: Most wireless noise-cancelling headphones come with rechargeable batteries that can provide hours of listening time on a single charge.\n\n4. Comfortable design: These headphones are typically designed to be lightweight and comfortable, with cush

## SimpleSequentialChain

In [76]:
from langchain.chains import SimpleSequentialChain

In [77]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Generate a detailed description of {product} including its main features and potential benefits."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [78]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description: {first_chain_output}, create a concise marketing tagline and list three customer personas who would benefit most from this product."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [79]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [80]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Wireless noise-cancelling headphones are a type of headphone that uses advanced technology to actively reduce or eliminate external noise, allowing the user to enjoy their music or audio content with crystal-clear sound quality. These headphones connect to devices such as smartphones, laptops, or tablets via Bluetooth, eliminating the need for tangled cords and providing the user with the freedom to move around without being tethered to their device.

One of the main features of wireless noise-cancelling headphones is their ability to block out background noise, such as traffic, airplane engine noise, or chatter in a crowded room. This is achieved through the use of built-in microphones that pick up ambient noise and create sound waves that cancel out these external sounds, allowing the user to focus on their music or audio content without distraction.

Another key feature of wireless noise-cancelling headphones is their superior sound qu

'Tagline: "Immerse Yourself in Crystal-Clear Sound Anywhere with Wireless Noise-Cancelling Headphones"\n\nCustomer Personas:\n1. Commuters: Busy individuals who rely on public transportation or have long commutes will benefit from the ability to block out noisy environments and enjoy their music or podcasts in peace.\n2. Travelers: Jetsetters and frequent flyers who want to escape the noise of airplanes and airports will appreciate the convenience and comfort of wireless noise-cancelling headphones.\n3. Fitness Enthusiasts: Active individuals who enjoy working out at the gym or jogging outdoors will love the freedom of movement that wireless headphones offer, along with the ability to focus on their workout without distractions.'

**Repeat the above twice for different products**

## SequentialChain

In [81]:
from langchain.chains import SequentialChain

In [82]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following customer review from {source_language} to {target_language}: {review_text}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )


In [83]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following customer review in 2-3 sentences, highlighting the main points, sentiment, and any specific product feedback: {translated_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_summary" #give a name to this output
                    )


In [84]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Detect the language of the following review and translate it to {target_language}: {review_text}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="final_review"
                      )


In [85]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Based on the translated review: {translated_review} and its summary: {review_summary}, generate a personalized response to the customer that addresses their main points and thanks them for their feedback."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="customer_response"
                     )


In [86]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["source_language", "target_language", "review_text"],
    output_variables=["translated_review", "review_summary", "customer_response"],
    verbose=True
)

In [87]:

review = df.Review[5]
result = overall_chain.invoke({
    "review_text": review,
    "source_language": "auto",  # Automatically detect the source language
    "target_language": "English"  # Translate to English
})
result



> Entering new SequentialChain chain...

> Finished chain.


{'review_text': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'source_language': 'auto',
 'target_language': 'English',
 'translated_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'review_summary': "The customer finds the taste of the product to be mediocre and is disappointed that the foam doesn't hold well. They suspect that the product may be old or counterfeit because they have purchased the same product in stores with a much better taste.",
 'customer_response': 'Dear valued customer,\n\nThank you for taking the time to share your feedback with us. We are sorry to hear that you found the taste of our product to be mediocre and that the foam did not hold well. We take the quality of our products very seriously and are disappointed to hear about 

**Repeat the above twice for different products or reviews**

## Router Chain

In [88]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


biology_template = """ You are a distinguished biology expert.\
You have extensive knowledge of cellular processes, genetics, evolution, ecology,\
and all living systems from microscopic organisms to complex ecosystems.\
You excel at explaining complex biological concepts in an accessible manner,\
drawing connections between different biological systems and levels of organization.\
You are particularly skilled at describing biological mechanisms step by step,\
explaining both structure and function, and highlighting the elegant solutions\
that have evolved in nature to solve various challenges.\
Your explanations balance molecular details with broader biological significance.

Here is a question:
{input}"""

In [89]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "biology", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [90]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [91]:
llm = ChatOpenAI(temperature=0)

In [92]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [93]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [94]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [95]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [96]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [97]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [98]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [99]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why do all cells in the human body have DNA?'}
> Finished chain.


'All cells in the human body have DNA because DNA contains the genetic information that determines the characteristics and functions of each cell. DNA serves as the blueprint for the development, growth, and maintenance of an organism. It carries the instructions for protein synthesis, which is essential for the structure and function of cells. Additionally, DNA is responsible for passing on genetic traits from one generation to the next through the process of reproduction. Therefore, having DNA in all cells ensures that the genetic information is preserved and maintained throughout the body.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**